# Applied Data Science Capstone Project
## Week 3 assignment
##### Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

# Part A

## Create a dataframe of neighborhood data for Toronto, Canada: postal codes, borough names, and neighborhood names

## Import needed libraries

In [119]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

print("Imports done")

Imports done


## Get the neighborhood data from Wikipedia, which has a table of the boroughs, neighborhoods, and postal codes of Toronto


In [120]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

print("The response data has been retrieved")

The response data has been retrieved


## Use BeautifulSoup for parsing the response data.
##### Working assumption is that the table we want is the first table in the response.

In [121]:
soup = BeautifulSoup(response.content, 'lxml')
table = soup.find('table')

## Convert the HTML table to a Pandas DataFrame

In [122]:
df_tor = pd.read_html(str(table), header=0)[0]
df_tor.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


## Clean the data per the assignment specification

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Set the column names to 'PostalCode', 'Borough', and 'Neighborhood

In [123]:
column_headers = ['PostalCode', 'Borough', 'Neighborhood']
df_tor.columns = column_headers
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'.

In [124]:
# Remove rows where 'Borough' is 'Not assigned'. Don't forget to re-index
# the rows!
df_tor = df_tor[df_tor['Borough'] != 'Not assigned'].reset_index(drop=True)
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## If a cell has a borough but a 'Not assigned' neighborhood, then change the neighborhood to the name of its borough.

In [125]:
df_tor['Neighborhood'].loc[df_tor['Neighborhood'] == 'Not assigned'] = df_tor['Borough']
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Combine neighborhoods with the same postal code into a single row, with the neighborhood names separated by a comma and a space, e.g., 'Neighborhood1, Neighborhood2'.

In [126]:
# I sort by the values first, so the joined neighborhoods appear in alphabetic order.
df_tor = df_tor.sort_values(['PostalCode', 'Neighborhood']).groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

## Display the shape of the dataframe

In [127]:
df_tor.shape

(103, 3)

# Part B

## Add each neighborhood's geographical coordinates, latitude and longitude, to the dataframe

##### I was unable to make geocoder work. I found that the google provider would not return any results; the osm provider returned unreliable results; and other providers required creating an account or paying for the access. Therefore , I am using the CSV file that was provided with the project instructions. The CSV file is included on GitHub with this notebook.

## Load the CSV file that includes latitude and longitude for the Toronto postal codes.

In [128]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
print("coordinates.shape:", coordinates.shape)
coordinates.head()

coordinates.shape: (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge the coordinates dataframe into df_tor

In [129]:
df_tor = df_tor.merge(coordinates, left_on='PostalCode', right_on='Postal Code', how='left')

# The 'Postal Code' column was included in the merged data, but it is not needed.
df_tor.drop('Postal Code', axis=1, inplace=True)

df_tor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Remove any rows if there are null values for Latitude or Longitude.
##### For this project, I know that the input data matches correctly, but in general, if the geospatial coordinates are not available, then we likely cannot use the data.

In [130]:
print("Before dropping rows with NaN coordinates:", df_tor.shape)
df_tor.dropna(subset = ['Latitude', 'Longitude'], inplace=True)
df_tor.reset_index(drop=True)
print("After dropping rows with NaN coordinates:", df_tor.shape)

Before dropping rows with NaN coordinates: (103, 5)
After dropping rows with NaN coordinates: (103, 5)


## Examine the first and last rows to make sure the dataframe looks right.

In [131]:
df_tor.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [132]:
df_tor.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054
